In [1]:
import torch
import numpy as np
import pandas as pd

In [14]:
df = pd.read_csv("dataset/framingham.csv")
df.info()
df=df.fillna((df.min()+df.max())/2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4238 non-null   int64  
 1   age              4238 non-null   int64  
 2   education        4133 non-null   float64
 3   currentSmoker    4238 non-null   int64  
 4   cigsPerDay       4209 non-null   float64
 5   BPMeds           4185 non-null   float64
 6   prevalentStroke  4238 non-null   int64  
 7   prevalentHyp     4238 non-null   int64  
 8   diabetes         4238 non-null   int64  
 9   totChol          4188 non-null   float64
 10  sysBP            4238 non-null   float64
 11  diaBP            4238 non-null   float64
 12  BMI              4219 non-null   float64
 13  heartRate        4237 non-null   float64
 14  glucose          3850 non-null   float64
 15  TenYearCHD       4238 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 529.9 KB


In [15]:
def sigmoid(z):
    return 1/(1+np.exp(z))

In [16]:
def ReLU(z):
    return np.maximum(0,z)

In [17]:
def tanh(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))

In [18]:
def LeakyReLU(z):
    return np.maximum(0.01*z,z)

In [19]:
y=df.iloc[:,15].values.reshape(-1,1)
x=df.iloc[:,:15].values.reshape(-1,15)

In [20]:
print(x.shape)
n=len(df)
n=(int)(n*0.8)
print (n)
df=df.T
print (df.shape)

(4238, 15)
3390
(16, 4238)


In [21]:
x_train=df.iloc[:15,:n].values.reshape(15,-1)
y_train=df.iloc[15,:n].values.reshape(1,-1)
x_test=df.iloc[:15,n:].values.reshape(15,-1)
y_test=df.iloc[15,n:].values.reshape(1,-1)

In [22]:
print (x_train)

[[ 1.    0.    1.   ...  1.    1.    1.  ]
 [39.   46.   48.   ... 37.   59.   50.  ]
 [ 4.    2.    1.   ...  2.    1.    1.  ]
 ...
 [26.97 28.73 25.34 ... 22.79 31.08 24.45]
 [80.   95.   75.   ... 56.   70.   69.  ]
 [77.   76.   70.   ... 76.   70.   70.  ]]


In [55]:
#init
w1=np.random.rand(2,15)*0.01
w2=np.random.rand(1,2)*0.01
#layers=2 
#layer hidden has 2 nodes
#layer output has 1 nodes
b1=np.zeros((2,1))
b2=np.zeros((1,1))


In [56]:
lr=0.001;

for i in range(1000):
    #forward
    z1=np.dot(w1,x_train)+b1
    a1=sigmoid(z1)
    z2=np.dot(w2,a1)+b2
    a2=sigmoid(z2)
    #cal loss
    #backward
    dz2=a2-y_train
    dw2=1/n*np.dot(dz2,a1.T)
    db2=1/n*np.sum(dz2,axis=1,keepdims=True)
    dz1=np.dot(w2.T,dz2)*(a1*(1-a1))
    dw1=1/n*np.dot(dz1,x_train.T)
    db1=1/n*np.sum(dz1,axis=1,keepdims=True)
    #update 
    w2=w2-lr*dw2
    b2=b2-lr*db2
    w1=w1-lr*dw1
    b1=b1-lr*db1

In [57]:
#evaluate
z1_pred=np.dot(w1,x_test)+b1
a1_pred=z1_pred
z2_pred=np.dot(w2,a1_pred)+b2
a2_pred=sigmoid(z2_pred)
y_pred=np.round(a2_pred)
print(y_pred)
res=abs(y_test-y_pred)
count =(1-((np.sum(res))/len(res.T)))*100
print("correct prediction:",count,"%")

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 